In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
from numpy.fft import fft, ifft
from scipy.signal import find_peaks
from scipy import signal
from scipy.signal import argrelextrema
py.init_notebook_mode(connected=True)
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
#from catboost import CatBoostClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('/content/drive/MyDrive/PCOMP/dataSetPCOMP_time.csv')
df

,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,...,t_32,t_33,t_34,t_35,t_36,t_37,t_38,t_39,t_40,device
0,-0.0072,-0.0064,-0.0088,-0.0072,-0.0072,-0.0064,-0.0048,-0.0032,-0.0024,0.0024,...,0.0016,0.0008,-0.0008,-0.0032,-0.0032,-0.0048,-0.0064,-0.0064,-0.0096,fan
1,-8.0000,-8.0000,-8.0000,-0.0104,-0.0072,-0.0064,-0.0064,-0.0048,-0.0032,-0.0024,...,4.0000,0.0024,0.0008,-0.0016,-0.0016,-0.0032,-4.0000,-0.0056,-0.0064,fan
2,-8.0000,-0.0072,-0.0088,-0.0072,-8.0000,-8.0000,-0.0072,-0.0064,-0.0064,-4.0000,...,0.0056,0.0048,0.0016,0.0024,0.0008,0.0000,-0.0016,-0.0032,-0.0048,fan
3,-0.0056,-0.0056,-8.0000,-0.0088,-8.0000,-8.0000,-8.0000,-0.0096,-0.0064,-0.0064,...,0.0088,0.0072,0.0056,0.0048,0.0032,0.0024,-0.0008,-0.0032,-0.0032,fan
4,-4.0000,-0.0056,-0.0064,-0.0064,-8.0000,-0.0088,-0.0088,-8.0000,-8.0000,-0.0072,...,12.0000,0.0096,8.0000,0.0072,0.0088,4.0000,0.0024,0.0008,-0.0008,fan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,...,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,airConditioner
4996,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,936.0000,752.0000,568.0000,352.0000,192.0000,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,airConditioner
4997,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,...,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,airConditioner
4998,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,-1.0000,...,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,1016.0000,airConditioner


In [3]:
classes = ["microwave", "airConditioner", "refrigerator", "monitor", "fan"]
X = df.drop(columns=['device'])
y = df['device'].apply(classes.index).to_numpy()
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

### KNN

In [4]:
GRID = [
    {'scaler': [StandardScaler()],
     'estimator': [KNeighborsClassifier()],
     'estimator__n_neighbors': [3, 5, 7],
     'estimator__weights': ['uniform', 'distance'],
     'estimator__p': [1, 2],
     'estimator__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
     'estimator__leaf_size': [30, 60, 90],
     'estimator__metric': ['euclidean', 'manhattan'],
     'estimator__n_jobs': [-1],
    }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', KNeighborsClassifier())])

In [5]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10, verbose=2)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
[CV] END estimator=KNeighborsClassifier(), estimator__algorithm=auto, estimator__leaf_size=30, estimator__metric=euclidean, estimator__n_jobs=-1, estimator__n_neighbors=3, estimator__p=1, estimator__weights=uniform, scaler=StandardScaler(); total time=   0.4s
[CV] END estimator=KNeighborsClassifier(), estimator__algorithm=auto, estimator__leaf_size=30, estimator__metric=euclidean, estimator__n_jobs=-1, estimator__n_neighbors=3, estimator__p=1, estimator__weights=uniform, scaler=StandardScaler(); total time=   0.1s
[CV] END estimator=KNeighborsClassifier(), estimator__algorithm=auto, estimator__leaf_size=30, estimator__metric=euclidean, estimator__n_jobs=-1, estimator__n_neighbors=3, estimator__p=1, estimator__weights=uniform, scaler=StandardScaler(); total time=   0.0s
[CV] END estimator=KNeighborsClassifier(), estimator__algorithm=auto, estimator__leaf_size=30, estimator__metric=euclidean, estimator__n_jobs=-1, estimator

In [6]:
best_model.best_params_

{'estimator': KNeighborsClassifier(metric='manhattan', n_jobs=-1, n_neighbors=3, p=1,
                      weights='distance'),
 'estimator__algorithm': 'auto',
 'estimator__leaf_size': 30,
 'estimator__metric': 'manhattan',
 'estimator__n_jobs': -1,
 'estimator__n_neighbors': 3,
 'estimator__p': 1,
 'estimator__weights': 'distance',
 'scaler': StandardScaler()}

In [7]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.8045938753832875
Acurácia:  0.824


## SVM

In [8]:
GRID = [
    {
        'scaler': [StandardScaler()],
        'estimator': [SVC(random_state=42)],
        'estimator__C': [0.1, 1, 10],
        'estimator__kernel': ['linear', 'rbf'],
        'estimator__gamma': [0.001, 0.01, 0.1, 1],
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', SVC())])

In [9]:
grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, scoring="f1_macro", cv=10, verbose=2)
best_model = grid_search.fit(x_train, y_train)
print('Melhor F1-SCORE: ', best_model.best_score_)

Fitting 10 folds for each of 24 candidates, totalling 240 fits
[CV] END estimator=SVC(random_state=42), estimator__C=0.1, estimator__gamma=0.001, estimator__kernel=linear, scaler=StandardScaler(); total time=   0.6s
[CV] END estimator=SVC(random_state=42), estimator__C=0.1, estimator__gamma=0.001, estimator__kernel=linear, scaler=StandardScaler(); total time=   0.6s
[CV] END estimator=SVC(random_state=42), estimator__C=0.1, estimator__gamma=0.001, estimator__kernel=linear, scaler=StandardScaler(); total time=   0.6s
[CV] END estimator=SVC(random_state=42), estimator__C=0.1, estimator__gamma=0.001, estimator__kernel=linear, scaler=StandardScaler(); total time=   0.6s
[CV] END estimator=SVC(random_state=42), estimator__C=0.1, estimator__gamma=0.001, estimator__kernel=linear, scaler=StandardScaler(); total time=   0.6s
[CV] END estimator=SVC(random_state=42), estimator__C=0.1, estimator__gamma=0.001, estimator__kernel=linear, scaler=StandardScaler(); total time=   0.6s
[CV] END estimator=

In [10]:
best_model.best_params_

{'estimator': SVC(C=10, gamma=1, random_state=42),
 'estimator__C': 10,
 'estimator__gamma': 1,
 'estimator__kernel': 'rbf',
 'scaler': StandardScaler()}

In [11]:
best_return = best_model.predict(x_test)
print("F1-Score: ",f1_score(y_test, best_return, average='macro'))
print("Acurácia: ", accuracy_score(y_test,best_return))

F1-Score:  0.6398561360729765
Acurácia:  0.6585
